# Lets Import the Necessary Packages

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.layers import Conv2D, Input, ZeroPadding2D
from keras.layers import BatchNormalization, Activation, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import RMSprop
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential
from keras.metrics import Accuracy
from keras import utils
import imutils
import numpy as np

# Data Augmentation

In [2]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import matplotlib as mt

train_datagen = ImageDataGenerator(rotation_range=40,width_shift_range=.2,height_shift_range=.2,rescale=1./255,
                                  shear_range=.2,zoom_range=.2,horizontal_flip=True,fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Read The Train Directory of images

In [5]:
batch_size = 10
train_data = train_datagen.flow_from_directory('/Users/gowthamshashank/downloads/Face_Detection/data-master/train',
                                              target_size = (150,150),batch_size=batch_size,
                                              class_mode='binary')

Found 1178 images belonging to 2 classes.


# Read The Test Directory of images

In [6]:
test_data = test_datagen.flow_from_directory('/Users/gowthamshashank/downloads/Face_Detection/data-master/test/',
                                            target_size=(150,150),batch_size=batch_size,
                                            class_mode='binary')

Found 194 images belonging to 2 classes.


# CNN Model Design

In [7]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 100)     2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 129600)            0         
_________________________________________________________________
dropout (Dropout)            (None, 129600)            0         
_________________________________________________________________
dense (Dense)                (None, 50)                6

# Define the compiler of cnn

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Train the model with the data

In [9]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',
                             verbose=0,save_best_only=True,mode='auto')

history = model.fit_generator(train_data,steps_per_epoch=50//batch_size,epochs=10,
                             validation_data=test_data,validation_steps=40//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
5/5 [==============================] - 3s 673ms/step - loss: 1.7161 - acc: 0.4400 - val_loss: 0.6932 - val_acc: 0.5250
Epoch 2/10
5/5 [==============================] - 3s 633ms/step - loss: 0.6933 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.4250
Epoch 3/10
5/5 [==============================] - 3s 631ms/step - loss: 0.6932 - acc: 0.3800 - val_loss: 0.6932 - val_acc: 0.4000
Epoch 4/10
5/5 [==============================] - 3s 628ms/step - loss: 0.6932 - acc: 0.4000 - val_loss: 0.6932 - val_acc: 0.4500
Epoch 5/10
5/5 [==============================] - 3s 620ms/step - loss: 0.6932 - acc: 0.4400 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 6/10
5/5 [==============================] - 3s 617ms/step - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.5250
Epoch 7/10
5/5 [==============================] - 3s 619ms/step - loss: 0.6932 - acc: 0.4400 - val_loss: 0.6932 - val_acc: 0.5500
Epoch 8/10
5/5

# Save trained model

In [10]:
model.save('mask_project.h5')

# Implement Face Mask detection with opencv - Haar cascade  classifier

In [11]:
import cv2
import numpy as np
from  keras.models import load_model

In [12]:
import tensorflow as tf
model = tf.keras.models.load_model('mask_project.h5')

In [13]:
result={0:'without mask', 1:'mask'}

In [14]:
result

{0: 'without mask', 1: 'mask'}

In [ ]:
import cv2
import numpy as np
from  keras.models import load_model
import tensorflow as tf
model = tf.keras.models.load_model('mask_project.h5')
result={0:'without mask', 1:'mask'}

GR_dict = {0:(0,0,255),1:(0,255,0)}

rect_size = 4

cap = cv2.VideoCapture(0)
haarcascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,30))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, result[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
